# Markowitz

In [ ]:
import numpy as np
import pandas as pd
import cvxopt
import os
import json
import cvxopt as opt
from cvxopt import blas, solvers
from cvxopt import matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# Hyperparameters #
base_folder='/content/drive/MyDrive/SEM 10/MTP Final/MTP 2/Outputs/LSTM_StockPicking_window_5'
target_return=0.2
out_folder='summary/lstm'
!gdown --id 1dkIQbawpE7BKnxGh_OTKI1qVHWCuhCXh # mtp_data.csv

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1dkIQbawpE7BKnxGh_OTKI1qVHWCuhCXh
To: /content/mtp_data.csv
100% 851k/851k [00:00<00:00, 103MB/s]


In [ ]:
def arr_to_ret(X):
	R = np.copy(X)
	for i in range(1, R.shape[0]):
		R[i] = (X[i] - X[i-1])/X[i-1]

	R = R[1:, :]
	return R

def markowitz_no_constraint(X):
    R = np.copy(X)
    for i in range(1, R.shape[0]):
        R[i] = (X[i] - X[i-1])/X[i-1]

    R = R[1:, :]

    # Solve optimiation problem #
    R_m = R - np.mean(R, axis=0)
    omega = np.dot(R_m.T, R_m)/(np.shape(R_m)[0])

    # No target return constraint #
    nAssets = omega.shape[0]
    e = np.ones((nAssets, 1))
    u = np.dot(np.linalg.inv(omega), e)
    w_opt = u/(np.dot(e.T, u))
    assert np.allclose(np.sum(w_opt), 1.0), "w_opt not summing to 1.0"
    mu = np.mean(R, axis=0).reshape(-1, 1)
    
    portfolio_return = np.sum(np.dot(mu.T, w_opt))
    portfolio_risk = np.dot(w_opt.T, np.dot(omega, w_opt))[0][0]
    sharpe_ratio=portfolio_return/portfolio_risk
    
    return portfolio_return, portfolio_risk, sharpe_ratio, w_opt

def markowitz_target_return(X, target_return):
    R = np.copy(X)
    for i in range(1, R.shape[0]):
        R[i] = (X[i] - X[i-1])/X[i-1]

    R = R[1:, :]

    # Solve optimiation problem #
    R_m = R - np.mean(R, axis=0)
    omega = np.dot(R_m.T, R_m)/(np.shape(R_m)[0])
    
    """
    1/2 x'Px + q'x
    subject to :
    Gx <= h
    Ax = b

    sol = solvers.qp(P,q,G,h,A,b)
    """
    nAssets = omega.shape[0]
    e = np.ones(nAssets).reshape(1, -1)
    mu = np.mean(R, axis=0).reshape(1, -1)

    P = matrix(2.0 * omega, tc='d') # tc='d' for double matrices
    q = matrix(np.zeros(nAssets))
    G = matrix(np.zeros((nAssets, nAssets)), tc='d')
    h = matrix(np.zeros(nAssets), tc='d')
    A = matrix(np.vstack([e, mu]), tc='d')
    b = matrix([1.0, target_return])

    sol = solvers.qp(P,q,G,h,A,b)

    w_opt = np.array(sol['x'])

    assert np.allclose(np.sum(w_opt), 1.0), "w_opt not summing to 1.0"
    assert np.allclose(np.dot(mu, w_opt), target_return), "mu'w not 0.2"
    
    portfolio_return = np.sum(np.dot(mu, w_opt))
    portfolio_risk = np.dot(w_opt.T, np.dot(omega, w_opt))[0][0]
    sharpe_ratio=portfolio_return/portfolio_risk
    
    return portfolio_return, portfolio_risk, sharpe_ratio, w_opt

In [ ]:
if not os.path.exists(out_folder):
	os.makedirs(out_folder)

In [ ]:
returns_p=[]
risks_p=[]
srs_p=[]
returns_actual=[]
risks_actual=[]
srs_actual=[]

weights_all_preds=[]
weights_all_true=[]

folder_names = [2, 4, 6, 8, 10]

for folder_name in folder_names:
	folder=os.path.join(base_folder, str(folder_name))

	with open(os.path.join(folder, 'commandline_args.txt')) as file:
		d=json.loads(file.read())
	idxs=np.load(os.path.join(folder, 'top_idxs.npy'))
	num_stocks=d['num_stocks']
	idxs=idxs[:num_stocks]

	preds=np.load(os.path.join(folder, 'preds_array.npy'))
	data=np.array(pd.read_csv('mtp_data.csv', index_col=0).iloc[:, :])
	assert preds.shape==data.shape
	preds=preds[:, idxs]
	data=data[:, idxs]

	# Test data average return compuation #
	data, test_data = train_test_split(data, test_size=d['test_size'], shuffle=False)
	test_mu=np.mean(arr_to_ret(test_data), axis=0).reshape(1, num_stocks)

	_, risk, _, w_opt = markowitz_no_constraint(preds)
	ret=np.dot(test_mu, w_opt)[0][0]
	sr=ret/risk
	risks_p.append(risk)
	returns_p.append(ret)
	srs_p.append(sr)
	weights_all_preds.append(list(w_opt.T)[0])
	
	print(data.shape)
	_, risk, _, w_opt = markowitz_no_constraint(data)
	ret=np.dot(test_mu, w_opt)[0][0]
	sr=ret/risk
	risks_actual.append(risk)
	returns_actual.append(ret)
	srs_actual.append(sr)
	weights_all_true.append(list(w_opt.T)[0])

plt.figure()
plt.plot(folder_names, returns_p, label='rets_predicted', color='orange')
plt.plot(folder_names, returns_actual, label='rets_actual', color='blue')
plt.xticks(folder_names)
plt.legend(loc ="lower right")
plt.title('LSTM Returns')
plt.savefig(os.path.join(out_folder, 'returns.png'))

plt.figure()
plt.plot(folder_names, risks_p, label = 'risks_predicted', color='orange')
plt.plot(folder_names, risks_actual, label = 'risks_actual', color='blue')
plt.xticks(folder_names)
plt.legend(loc ="lower right")
plt.title('LSTM Risks')
plt.savefig(os.path.join(out_folder, 'risks.png'))

plt.figure()
plt.plot(folder_names, srs_p, label = 'SR_predicted', color='orange')
plt.plot(folder_names, srs_actual, label = 'SR_actual', color='blue')
plt.xticks(folder_names)
plt.legend(loc ="lower right")
plt.title('LSTM Sharpe Ratio')
plt.savefig(os.path.join(out_folder, 'sr.png'))

np.savetxt(out_folder + '/returns_p.txt', returns_p, fmt='%10.5f')
np.savetxt(out_folder + '/returns_actual.txt', returns_actual, fmt='%10.5f')
np.savetxt(out_folder + '/risks_p.txt', risks_p, fmt='%10.5f')
np.savetxt(out_folder + '/risks_actual.txt', risks_actual, fmt='%10.5f')
np.savetxt(out_folder + '/srs_p.txt', srs_p, fmt='%10.5f')
np.savetxt(out_folder + '/srs_actual.txt', srs_actual, fmt='%10.5f')
np.savetxt(out_folder + '/weights_all_preds.txt', weights_all_preds, fmt='%10.5f')
np.savetxt(out_folder + '/weights_all_true.txt', weights_all_true, fmt='%10.5f')

# with open(os.path.join(out_folder, 'results.txt'), 'w') as f:
#   f.write('Predicted Returns for test sizes from 2 to 10: ' + str(returns_p))
#   f.write('\n Actual Returns for test sizes from 2 to 10: ' + str(returns_actual))
#   f.write('\n Predicted Variance for test sizes from 2 to 10: ' + str(risks_p))
#   f.write('\n Actual Variance for test sizes from 2 to 10: ' + str(risks_actual))
#   f.write('\n Predicted Sharpe Ratio for test sizes from 2 to 10: ' + str(srs_p))
#   f.write('\n Actual Sharpe Ratio for test sizes from 2 to 10: ' + str(srs_actual))
# 	# f.write('\n Weights Predicted for test sizes from 1 to 10: ' + srt(weights_all_preds))
# 	# f.write('\n Weights Actual for test sizes from 1 to 10: ' + srt(weights_all_true))

In [ ]:
# Downloading outputs
!zip -r /content/markowitz_output.zip /content/summary